In [1]:
from google.cloud import vision
import pandas as pd

In [2]:
import os
import glob
import base64

In [3]:
import io
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types
import json
from google.protobuf.json_format import MessageToJson




from fnmatch import fnmatch


client = vision.ImageAnnotatorClient()
image = vision.types.Image()

root = '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/'
pattern = "*.png"

cols = ['AdId','num_faces','face_joylikelihood', 'face_sorrowLikelihood', 'face_angerLikelihood','face_surpriseLikelihood','face_description_confidence',
                    'landmark_features','labels_description','labels_descriptions_confidence','logo_description','logo_description_confidence','text_descriptions',
                   'object_descriptions','object_descriptions_confidence','isAdult','isSpoof','isMedical','isViolence','isRacy']
df =  pd.DataFrame(columns = cols)  



for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            #print os.path.join(path, name)
            print (path + "/" + name)

            #client = vision.ImageAnnotatorClient()
            #image_path = '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png'

            # Loads the image into memory
            filename = (path  + "/" + name)
            #filename =  '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png'
            with io.open(filename, 'rb') as image_file:
                content = image_file.read()

            image = types.Image(content=content)


            query = {

                "image": image ,

                  "features": [
                    {
                      "type": "LANDMARK_DETECTION"
                    },
                    {
                      "type": "FACE_DETECTION"
                    },
                    {
                      "type": "OBJECT_LOCALIZATION"
                    },
                    {
                      "type": "LOGO_DETECTION"
                    },
                    {
                      "type": "LABEL_DETECTION"
                    },
                    {
                      "type": "DOCUMENT_TEXT_DETECTION"
                    },
                    {
                      "type": "SAFE_SEARCH_DETECTION"
                    },
                    {
                      "type": "IMAGE_PROPERTIES"
                    },
                    {
                      "type": "CROP_HINTS"
                    },
                    {
                      "type": "WEB_DETECTION"
                    }
                  ],

                }



            response = client.annotate_image(query)
            #print(type(response))
            
            #response_json = MessageToJson(response)
            #print(type(response_json))
            
            # FACE ANNOTATIONS
            joylikelihood = []
            sorrowLikelihood = []
            angerLikelihood = []
            surpriseLikelihood = []
            face_description_score = []
            num_faces = len(response.face_annotations)
            for i in range(len(response.face_annotations)):
                joylikelihood.append(vision.enums.Likelihood(response.face_annotations[i].joy_likelihood).name)
                sorrowLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].sorrow_likelihood).name)
                angerLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].anger_likelihood).name)
                surpriseLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].surprise_likelihood).name)
                face_description_score.append(response.face_annotations[i].detection_confidence)
            
            #Landmark Features
            #response.landmark_annotations
            landmarkFeatures = []
            for i in range(len(response.landmark_annotations)):
                landmarkFeatures.append(str(response.landmark_annotations[i].description))
            
            #Label Features
            labels_description = []
            labels_scores = []
            for i in range(len(response.label_annotations)):
                labels_description.append(str(response.label_annotations[i].description))
                labels_scores.append(response.label_annotations[i].score)
                
            #Logo Descriptions
            logo_description = []
            logo_description_score = []
            for i in range(len(response.logo_annotations)):
                logo_description.append(str(response.logo_annotations[i].description.encode('utf-8')))
                logo_description_score.append(response.logo_annotations[i].score)
                
            #TEXT Features    
            text_descriptions = []
            for i in range(len(response.text_annotations)):
                if(len(response.text_annotations[i].description) > 2):
                    text_descriptions.append(str(response.text_annotations[i].description.encode('utf-8')))
                
            #Safe search
            isAdult = vision.enums.Likelihood(response.safe_search_annotation.adult).name
            isSpoof = vision.enums.Likelihood(response.safe_search_annotation.spoof).name
            isMedical= vision.enums.Likelihood(response.safe_search_annotation.medical).name
            isViolence = vision.enums.Likelihood(response.safe_search_annotation.violence).name
            isRacy = vision.enums.Likelihood(response.safe_search_annotation.racy).name
            
            #Object description
            objects_descriptions = []
            object_score = []
            for i in range(len(response.localized_object_annotations)):
                objects_descriptions.append(str(response.localized_object_annotations[i].name))
                object_score.append(response.localized_object_annotations[i].score)
            
            
            ADId = "A" + str(path.split(os.sep)[-1]).zfill(2) + "_" + str(os.path.splitext(name)[0]).zfill(2)
            #print (path.split(os.sep))
            print (ADId)
            
            df = df.append({'AdId' : ADId ,
                            'num_faces' : num_faces,
                            'face_joylikelihood': joylikelihood , 
                            'face_sorrowLikelihood' : sorrowLikelihood , 
                            'face_angerLikelihood' : angerLikelihood ,
                            'face_surpriseLikelihood' : surpriseLikelihood ,
                            'face_description_confidence': face_description_score,
                            'landmark_features' : landmarkFeatures,
                            'labels_description' : labels_description,
                            'labels_descriptions_confidence' : labels_scores , 
                            'logo_description' : logo_description , 
                            'logo_description_confidence' : logo_description_score , 
                            'text_descriptions' : text_descriptions,
                            'object_descriptions' : objects_descriptions,
                            'object_descriptions_confidence': object_score,
                            'isAdult' : isAdult , 
                            'isSpoof':isSpoof,'isMedical':isMedical,'isViolence':isViolence,'isRacy':isRacy
                           }, ignore_index=True)

/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/8.png
A20_08
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/9.png
A20_09
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/14.png
A20_14
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/15.png
A20_15
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/12.png
A20_12
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png
A20_13
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/11.png
A20_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/10.png
A20_10
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/4.png
A20_04
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/5.png
A

A16_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/10.png
A16_10
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/4.png
A16_04
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/5.png
A16_05
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/7.png
A16_07
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/6.png
A16_06
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/2.png
A16_02
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/3.png
A16_03
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/1.png
A16_01
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/6/8.png
A06_08
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/6/9.png
A

A19_02
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/19/3.png
A19_03
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/19/1.png
A19_01
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/8.png
A08_08
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/9.png
A08_09
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/14.png
A08_14
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/15.png
A08_15
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/12.png
A08_12
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/13.png
A08_13
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/11.png
A08_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/8/10.png
A0

A02_13
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/11.png
A02_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/10.png
A02_10
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/4.png
A02_04
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/5.png
A02_05
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/7.png
A02_07
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/6.png
A02_06
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/2.png
A02_02
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/3.png
A02_03
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/2/1.png
A02_01
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/13/8.png
A13_08


In [6]:
    #response
    df
    

,AdId,num_faces,face_joylikelihood,face_sorrowLikelihood,face_angerLikelihood,face_surpriseLikelihood,face_description_confidence,landmark_features,labels_description,labels_descriptions_confidence,logo_description,logo_description_confidence,text_descriptions,object_descriptions,object_descriptions_confidence,isAdult,isSpoof,isMedical,isViolence,isRacy
0,A20_08,0,[],[],[],[],[],[],"[Text, Font, Logo, Brand]","[0.954084157944, 0.831454455853, 0.79397326707...",[],[],"[iviy\nTM\nMySocialCircle.net - MEET\nPEOPLE, ...",[],[],VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
1,A20_09,2,"[VERY_UNLIKELY, VERY_UNLIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[0.945869207382, 0.817052721977]",[],"[Text, Font, Photography]","[0.869291961193, 0.685723781586, 0.624224960804]",[],[],[Online Dating Tips and Advices 17\nJul 2013\n...,"[Person, Person, Person, Clothing, Clothing]","[0.72792416811, 0.65952372551, 0.655169188976,...",VERY_UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY
2,A20_14,1,[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[0.897761583328],[],"[Product, Mobile phone, Gadget, Smartphone, Co...","[0.955397307873, 0.950451731682, 0.94890356063...",[],[],[tinder\ntinder\nDiscover those around you.\nT...,"[Mobile phone, Mobile phone, Person]","[0.89793920517, 0.894875526428, 0.862094819546]",UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY
3,A20_15,1,[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[0.969707489014],[],"[Product, Text, Font, Line, Magenta, Technolog...","[0.94625878334, 0.932497680187, 0.804016649723...",[],[],[LOVOO @lovoo\n13 Jun\n12 Millionen Nutzer. Ch...,"[Packaged goods, Person]","[0.708368718624, 0.517164349556]",VERY_UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY
4,A20_12,1,[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[0.956528902054],[],"[Skin, Beauty, Text, Font, Dress, Headgear, Fa...","[0.913795828819, 0.882392942905, 0.88115477562...","[Bebe Stores, Sur La Table]","[0.683087229729, 0.669328868389]",[badoo\nMEET NEW PEOPLE.\n150 MILLION OF THEM....,"[Person, Hat, Dress]","[0.905352294445, 0.844507992268, 0.667487502098]",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,POSSIBLE
5,A20_13,2,"[VERY_LIKELY, VERY_LIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[VERY_UNLIKELY, VERY_UNLIKELY]","[0.82688242197, 0.800904631615]",[],"[Product, Text, Child, Advertising, Font, Phot...","[0.957412600517, 0.89013081789, 0.711644530296...",[eHarmony],[0.933201432228],[e Harmony\nCompatible\nPeople...\nGreat\nRela...,"[Person, Clothing]","[0.777252197266, 0.557873785496]",VERY_UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
6,A20_11,1,[VERY_LIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[VERY_UNLIKELY],[0.898022353649],[],"[Skin, Text, Head, Forehead, Chin, Cheek, Font...","[0.919962823391, 0.89013081789, 0.876235961914...",[Bebe Stores],[0.670205593109],[Badoo\nMEET NEW PEOPLE.\n150 MILLION OF THEM\...,"[Top, Person]","[0.817453145981, 0.792627334595]",VERY_UNLIKELY,POSSIBLE,UNLIKELY,UNLIKELY,POSSIBLE
7,A20_10,0,[],[],[],[],[],[],"[Text, Font, Logo, Heart, Poster, Graphic desi...","[0.928794205189, 0.861624836922, 0.74367809295...",[],[],[Find Love\nDating Sites 1 Nov 2011\nBryan Rob...,[],[],VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
8,A20_04,0,[],[],[],[],[],[],"[Text, Font, Line, Logo]","[0.975615918636, 0.890847146511, 0.82264310121...",[],[],[Senior Dating Site\nwww.maturefreeandsingle.c...,[],[],VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
9,A20_05,0,[],[],[],[],[],[],"[Text, Line, Font]","[0.977747559547, 0.864646017551, 0.861624836922]",[],[],"[Meet New People on Badoo, Make Friends, Chat,...",[],[],VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY


In [7]:
    df.to_csv('Ads_images_annotated.csv')

# Explore 

## Face Features

In [30]:
    #response.face_annotations[0]
    

IndexError: list index (0) out of range

In [33]:
    #response.face_annotations[0].joy_likelihood
    #print((response.face_annotations))
    joylikelihood = []
    sorrowLikelihood = []
    angerLikelihood = []
    surpriseLikelihood = []
    for i in range(len(response.face_annotations)):
        joylikelihood.append(response.face_annotations[i].joy_likelihood)
        sorrowLikelihood.append(response.face_annotations[i].sorrow_likelihood)
        angerLikelihood.append(response.face_annotations[i].anger_likelihood)
        surpriseLikelihood.append(response.face_annotations[i].surprise_likelihood)


[]


## Landmark Features

In [40]:
    response.landmark_annotations
    landmarkFeatures = []
    for i in range(len(response.landmark_annotations)):
        landmarkFeatures.append(response.landmark_annotations[i].description)
    

In [35]:
    response.landmark_annotations

[]

## Label Features

In [13]:
    labels_description = []
    labels_scores = []
    for i in range(len(response.label_annotations)):
        labels_description.append(response.label_annotations[i].description)
        labels_scores.append(response.label_annotations[i].score)
        
    print(labels_scores)

[0.9708330035209656, 0.9509816765785217, 0.6661911010742188, 0.5848097205162048, 0.5733055472373962, 0.5518041253089905]


## Text Annotation

In [8]:
    response.text_annotations[0].description
    text_descriptions = []
    for i in range(len(response.text_annotations)):
        text_descriptions.append(response.text_annotations[i].description)
        
    (str(text_descriptions[0]))
    

'Blue Shoes - 25% Off\nFree Shipping, Exchanges & Returns\nBlue Shoes at anesshoes.com! Code: BLUE\nwww.anesshoes.com/colors\n'

## Safe search


In [123]:
    response.safe_search_annotation

adult: VERY_UNLIKELY
spoof: VERY_UNLIKELY
medical: VERY_UNLIKELY
violence: VERY_UNLIKELY
racy: VERY_UNLIKELY

In [121]:
    
    vision.enums.Likelihood(response.safe_search_annotation.adult).name
    vision.enums.Likelihood(response.safe_search_annotation.spoof).name
    vision.enums.Likelihood(response.safe_search_annotation.medical).name
    vision.enums.Likelihood(response.safe_search_annotation.violence).name
    vision.enums.Likelihood(response.safe_search_annotation.racy).name
    
    
    

'VERY_UNLIKELY'

## Image properties

In [21]:
    #response.dominant_colors_annotation